In [3]:
import numpy as np

In [4]:
def fair_dice():
    return np.random.choice(np.arange(1, 7), 1)[0]

def fake_dice():
    return np.random.choice(np.arange(1, 7), 1, p=[0.1, 0.1, 0.1, 0.1, 0.1, 0.5])[0]

In [145]:
def rolls(N, p0, p1):
    res = np.zeros(N, dtype=int)
    truth = []
    dice = True
    for i in range(N):
        if dice and np.random.random() < p0:
            dice = False
        if not dice and np.random.random() < p1:
            dice = True
        if dice:
            res[i] = fair_dice()
            truth.append(0)
        else:
            res[i] = fake_dice()
            truth.append(1)
    return (res, truth)

In [146]:
def subsequences(xs, n):
    return (xs[i:i+n] for i in range(len(xs)))

In [163]:
def heuristic(res):
    guess = np.zeros(len(res), dtype=int)
    idx = -1
    for seq in subsequences(res, 10):
        idx += 1
        if np.count_nonzero(seq == 6) > 3:
            if idx + 10 < len(res):
                for x in range(idx, idx+10):
                    guess[x] = 1
            else:
                for x in range(idx, len(res)):
                    guess[x] = 1
    return guess.tolist()

In [164]:
def measure(a, b):
    return len([i for i, j in zip(a, b) if i == j]) / len(a)

In [169]:
def hmm(res):
    N = res.shape[0]
    A = np.array([[0.96, 0.04], [0.05, 0.95]])
    B = np.array([[1/6, 1/6, 1/6, 1/6, 1/6, 1/6], [1/10, 1/10, 1/10, 1/10, 1/10, 1/2]])
    alphas = np.zeros((A.shape[0], N))
    betas = np.zeros((A.shape[0], N))
    
    alphas[0, 0] = 1
    alphas[1, 0] = 0
    
    for i in range(1, N):
        for j in range(2):
            for k in range(2):
                alphas[j, i] += alphas[k, i-1] * A[k, j] * B[k, res[i]-1]
            alphas[j, i] *= 5.5
    
    betas[:, N-1] = 1
    
    for i in reversed(range(N-1)):
        for j in range(2):
            for k in range(2):
                betas[j, i] += betas[k, i+1] * A[j, k] * B[k, res[i]-1]
            betas[j,i] *= 5.5
    
    gammas = alphas * betas / np.sum(alphas * betas, axis=0)
    guess = np.argmax(gammas, axis=0)
    return guess.tolist()

In [188]:
test = rolls(10000, 0.04, 0.05)

In [189]:
print(test[0][:100])
print(test[1][:100])

[1 5 4 2 1 2 2 1 3 1 6 5 2 5 5 6 3 6 5 1 6 5 4 6 6 6 6 6 4 5 1 1 2 5 3 5 1
 6 6 3 2 6 2 6 6 5 3 6 6 6 6 6 6 5 6 6 2 5 6 6 6 5 1 6 4 2 6 1 6 2 6 5 1 3
 4 3 6 4 1 1 4 5 3 6 4 5 1 6 6 6 6 6 6 6 6 3 5 3 6 6]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [190]:
measure(test[1], heuristic(test[0]))

0.7601

In [191]:
measure(test[1], hmm(test[0]))

0.8322